In [1]:
from dune_client.client import DuneClient
dune = DuneClient("rVRuwpjjYaD1yXJAVH3dVZufZcESZK4E")
query_result = dune.get_latest_result(4705860)



In [3]:
import pandas as pd
# 提取 rows 数据
rows = query_result.result.rows

# 创建 DataFrame
df = pd.DataFrame(rows)

# 如果需要，将 transaction_date 列转换为 datetime 类型
df['transaction_time'] = pd.to_datetime(df['transaction_time'])

# 查看 DataFrame
df

,avg_value_per_transaction,total_gas_fee,total_unique_addresses,total_value,transaction_count,transaction_time,unique_from_addresses,unique_to_addresses
0,2017.025135,145233.446630,255897,2.905880e+09,1440676,2024-01-01 00:00:00+00:00,246275,9622
1,581.295820,66150.699119,370244,6.942736e+08,1194355,2024-01-02 00:00:00+00:00,362828,7416
2,670.637719,51581.905134,232915,6.549535e+08,976613,2024-01-03 00:00:00+00:00,226825,6090
3,396.472604,25110.079753,111322,2.953102e+08,744844,2024-01-04 00:00:00+00:00,104426,6896
4,150.557765,16727.767695,93226,1.073581e+08,713069,2024-01-05 00:00:00+00:00,87294,5932
...,...,...,...,...,...,...,...,...
432,64.532993,5143.608626,593540,6.710721e+07,1039890,2025-03-08 00:00:00+00:00,392848,200692
433,115.660109,6904.895651,537364,1.762573e+08,1523925,2025-03-09 00:00:00+00:00,398779,138585
434,88.360565,7548.902924,743505,1.349762e+08,1527562,2025-03-10 00:00:00+00:00,598338,145167
435,53.178707,7972.472463,729075,1.175062e+08,2209648,2025-03-11 00:00:00+00:00,638312,90763


In [4]:
df.to_csv('kaia.csv', index=False)


In [ ]:
import ccxt
import pandas as pd
import datetime

def get_kaia_historical_prices():
    # 初始化 Gate.io 交易所
    exchange = ccxt.gateio()

    # 设置交易对
    symbol = 'KAIA/USDT'  # 假设以 USDT 计价

    # 设置时间范围
    days_to_fetch = 300  # 总共获取 90 天的数据
    timeframe = '1d'  # 1天的时间框架
    all_data = []
    since = exchange.parse8601((datetime.datetime.now() - datetime.timedelta(days=days_to_fetch)).isoformat())

    # 分批获取数据
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
        if not ohlcv:
            break  # 如果没有数据则退出

        all_data.extend(ohlcv)  # 累积数据

        # 更新 since 为最后一个数据的时间戳 + 1 毫秒
        since = ohlcv[-1][0] + 1

    # 将结果转换为 DataFrame
    df = pd.DataFrame(all_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')  # 转换时间戳为日期
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]  # 重新排列列

    # 去除重复日期
    df = df.drop_duplicates(subset='date')

    return df  # 返回 DataFrame

# 调用函数并输出 DataFrame
df = get_kaia_historical_prices()
print(df)

Empty DataFrame
Columns: [date, open, high, low, close, volume]
Index: []


In [21]:
df.to_csv("kaia_price.csv")